In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth='day'
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=2*forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list(rawArrayDatas[1])
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list(rawArrayDatas[1][:-forecastDay] )
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-3*forecastDay] & np.log
    ds = rawArrayDatas[0][:-3*forecastDay]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-3*forecastDay]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of 2*forecastDay:  rawArrayDatas[1][-3*forecastDay:-forecastDay]
    testY= rawArrayDatas[1][-3*forecastDay:-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

        print('LSTM forecast :',mockForecastDictionary['LSTM'] )
        ####Bayseian

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        print('Bayseian forecast :',mockForecastDictionary['LSTM'] )
        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian']=Bayseian(txsForRealForecastBayesian,forecastDay,'day')

    elif dayOrWeekOrMonth is 'week':
        ####LSTM

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)

    elif dayOrWeekOrMonth is 'month':
        ####LSTM

        # select feature module
        feature = 'Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    



        ####Bayseian

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        ####더 좋은 알고리즘 호출
        if nameOfBestAlgorithm is 'LSTM':
            tf.reset_default_graph()
            realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)

        elif nameOfBestAlgorithm is 'Bayseian':
            realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')


        ####################################################################################BAYSEIAN
    # tf.reset_default_graph()
    # realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay, feature)

    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
    elif feature is 'Month_Season_Year':
        tempxy = [list(txs['month']), list(txs['season']), list(txs['year']), sales]
    xy = np.array(tempxy).transpose().astype(np.float)
    xy = np.array(tempxy).transpose().astype(np.float)

    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
    return test_predict[-1].tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm

    return nameOfBestAlgorithm



In [ ]:
def LSTM(txs, forecastDay, features)


In [ ]:
def Bayseian(txs, forecastDay, unit)

In [2]:
columns=['ds','y']
txs=pd.read_table('walMonth_train.csv', sep=',',header=None,names=columns )

In [4]:
txs

,ds,y
0,2010-02-01,32990.7700
1,2010-03-01,22809.2850
2,2010-04-01,30103.3520
3,2010-05-01,16673.5375
4,2010-06-01,16685.1750
5,2010-07-01,16383.0020
6,2010-08-01,16144.7025
7,2010-09-01,17978.3175
8,2010-10-01,26928.9060
9,2010-11-01,23040.3500


In [ ]:
# len(txs)
# LSTM(txs, 7, features)
# Bayseian(txs, 7, 'month')

In [8]:
rawArrayDatas=[ ['2007-12-01', '2008-01-01', '2008-02-01', '2008-03-01', '2008-04-01', '2008-05-01', '2008-06-01', '2008-07-01', '2008-08-01', '2008-09-01', '2008-10-01', '2008-11-01', '2008-12-01', '2009-01-01', '2009-02-01', '2009-03-01', '2009-04-01', '2009-05-01', '2009-06-01', '2009-07-01', '2009-08-01', '2009-09-01', '2009-10-01', '2009-11-01', '2009-12-01', '2010-01-01', '2010-02-01', '2010-03-01', '2010-04-01', '2010-05-01', '2010-06-01', '2010-07-01', '2010-08-01', '2010-09-01', '2010-10-01', '2010-11-01', '2010-12-01', '2011-01-01', '2011-02-01', '2011-03-01', '2011-04-01', '2011-05-01', '2011-06-01', '2011-07-01', '2011-08-01', '2011-09-01', '2011-10-01', '2011-11-01', '2011-12-01', '2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01', '2012-05-01', '2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01', '2016-01-01'] , [4958.0, 8359.2999999999993, 12178.14286, 1735.5357140000001, 2178.5666670000001, 1424.419355, 1237.7142859999999, 1343.4545449999998, 2100.580645, 3135.5999999999999, 2644.517241, 2929.0, 2971.4838709999999, 4590.8709680000002, 2453.4642859999999, 1603.2903229999999, 1760.5, 1483.4838710000001, 1376.5333330000001, 1725.9354840000001, 2844.6451609999999, 5786.4615380000005, 6445.6428569999998, 6952.2142860000004, 7556.0322580000002, 13760.172409999999, 12469.23077, 2277.3225809999999, 2503.9666670000001, 1698.451613, 1425.5357140000001, 1848.6153850000001, 3838.3225810000004, 9605.4666670000006, 6985.6774189999996, 8732.5, 7899.0645159999995, 9540.9032260000004, 3907.1071430000002, 2085.8709680000002, 3015.5666670000001, 1886.193548, 1695.9000000000001, 3470.6451609999999, 4622.2258060000004, 9086.6896550000001, 6864.8666670000002, 7461.9666670000006, 7556.3103449999999, 19610.45161, 25008.793099999999, 24771.03226, 3943.3103450000003, 3136.3870969999998, 2406.8666670000002, 3127.6451609999999, 5017.7741939999996, 13273.733329999999, 9823.8064519999989, 7519.6666670000004, 9169.16129, 13726.903230000002, 4432.5, 2776.5161290000001, 2286.6333329999998, 1913.451613, 1650.9333329999999, 2040.0333329999999, 2734.0645159999999, 13740.266669999999, 10564.35484, 7608.3999999999996, 8706.1935480000011, 26366.56667, 24504.89286, 2169.0322579999997, 1879.0, 2449.5483870000003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] ]

In [9]:
forecastDay=int(len(rawArrayDatas[1])*0.2)+1

In [10]:
forecastDay

20

In [11]:
LearningModuleRunner(rawArrayDatas,7,forecastDay)

[step: 0] loss: 78.11444854736328
[step: 1] loss: 76.57946014404297
[step: 2] loss: 75.05852508544922
[step: 3] loss: 73.52033233642578
[step: 4] loss: 71.92718505859375
[step: 5] loss: 70.23699188232422
[step: 6] loss: 68.40721893310547
[step: 7] loss: 66.39246368408203
[step: 8] loss: 64.14389038085938
[step: 9] loss: 61.61175537109375
[step: 10] loss: 58.749969482421875
[step: 11] loss: 55.52494812011719
[step: 12] loss: 51.93253707885742
[step: 13] loss: 48.029212951660156
[step: 14] loss: 43.98804473876953
[step: 15] loss: 40.19749450683594
[step: 16] loss: 37.395626068115234
[step: 17] loss: 36.592567443847656
[step: 18] loss: 37.73310089111328
[step: 19] loss: 38.38792419433594
[step: 20] loss: 37.178863525390625
[step: 21] loss: 34.90205001831055
[step: 22] loss: 32.76488494873047
[step: 23] loss: 31.424392700195312
[step: 24] loss: 30.92165756225586
[step: 25] loss: 30.9703369140625
[step: 26] loss: 31.235294342041016
[step: 27] loss: 31.473222732543945
[step: 28] loss: 31.561

[step: 243] loss: 12.77586555480957
[step: 244] loss: 12.676218032836914
[step: 245] loss: 12.593024253845215
[step: 246] loss: 12.5343656539917
[step: 247] loss: 12.48331069946289
[step: 248] loss: 12.425052642822266
[step: 249] loss: 12.344083786010742
[step: 250] loss: 12.261795043945312
[step: 251] loss: 12.18899917602539
[step: 252] loss: 12.129934310913086
[step: 253] loss: 12.079739570617676
[step: 254] loss: 12.023150444030762
[step: 255] loss: 11.96636962890625
[step: 256] loss: 11.8942289352417
[step: 257] loss: 11.829404830932617
[step: 258] loss: 11.778553009033203
[step: 259] loss: 11.737251281738281
[step: 260] loss: 11.694928169250488
[step: 261] loss: 11.602041244506836
[step: 262] loss: 11.497659683227539
[step: 263] loss: 11.405752182006836
[step: 264] loss: 11.349365234375
[step: 265] loss: 11.311564445495605
[step: 266] loss: 11.252788543701172
[step: 267] loss: 11.177560806274414
[step: 268] loss: 11.086441993713379
[step: 269] loss: 11.01856517791748
[step: 270] l

[step: 483] loss: 2.9819672107696533
[step: 484] loss: 3.0314905643463135
[step: 485] loss: 3.1064653396606445
[step: 486] loss: 3.2871806621551514
[step: 487] loss: 3.433027744293213
[step: 488] loss: 3.8095364570617676
[step: 489] loss: 3.615952491760254
[step: 490] loss: 3.5132250785827637
[step: 491] loss: 3.054818630218506
[step: 492] loss: 2.8481626510620117
[step: 493] loss: 2.899414300918579
[step: 494] loss: 3.0861434936523438
[step: 495] loss: 3.304426670074463
[step: 496] loss: 3.155911445617676
[step: 497] loss: 2.9785280227661133
[step: 498] loss: 2.796476364135742
[step: 499] loss: 2.776444435119629
[step: 500] loss: 2.8809194564819336
[step: 501] loss: 2.9660964012145996
[step: 502] loss: 3.0134236812591553
[step: 503] loss: 2.88824200630188
[step: 504] loss: 2.7739076614379883
[step: 505] loss: 2.6928882598876953
[step: 506] loss: 2.6824822425842285
[step: 507] loss: 2.7257726192474365
[step: 508] loss: 2.7824201583862305
[step: 509] loss: 2.8584179878234863
[step: 510]

[step: 712] loss: 1.548382043838501
[step: 713] loss: 1.4984078407287598
[step: 714] loss: 1.3591487407684326
[step: 715] loss: 1.2255122661590576
[step: 716] loss: 1.2200692892074585
[step: 717] loss: 1.3078243732452393
[step: 718] loss: 1.365070104598999
[step: 719] loss: 1.3442481756210327
[step: 720] loss: 1.2495633363723755
[step: 721] loss: 1.1852834224700928
[step: 722] loss: 1.1890082359313965
[step: 723] loss: 1.2351765632629395
[step: 724] loss: 1.2738876342773438
[step: 725] loss: 1.2608426809310913
[step: 726] loss: 1.2192436456680298
[step: 727] loss: 1.1707830429077148
[step: 728] loss: 1.1469051837921143
[step: 729] loss: 1.1514177322387695
[step: 730] loss: 1.172203540802002
[step: 731] loss: 1.196118950843811
[step: 732] loss: 1.2083287239074707
[step: 733] loss: 1.213756799697876
[step: 734] loss: 1.20309317111969
[step: 735] loss: 1.1918853521347046
[step: 736] loss: 1.1739811897277832
[step: 737] loss: 1.1607987880706787
[step: 738] loss: 1.1475176811218262
[step: 7

[step: 950] loss: 0.6509965658187866
[step: 951] loss: 0.6321827173233032
[step: 952] loss: 0.5304298400878906
[step: 953] loss: 0.6061193346977234
[step: 954] loss: 0.5839805603027344
[step: 955] loss: 0.5380548238754272
[step: 956] loss: 0.5725740194320679
[step: 957] loss: 0.5448795557022095
[step: 958] loss: 0.5234650373458862
[step: 959] loss: 0.560758650302887
[step: 960] loss: 0.5190956592559814
[step: 961] loss: 0.5032616853713989
[step: 962] loss: 0.535912275314331
[step: 963] loss: 0.5136353969573975
[step: 964] loss: 0.4914332628250122
[step: 965] loss: 0.5101022720336914
[step: 966] loss: 0.49612441658973694
[step: 967] loss: 0.48865002393722534
[step: 968] loss: 0.49779775738716125
[step: 969] loss: 0.4822554588317871
[step: 970] loss: 0.47224509716033936
[step: 971] loss: 0.48574045300483704
[step: 972] loss: 0.47791996598243713
[step: 973] loss: 0.4668048620223999
[step: 974] loss: 0.4702133238315582
[step: 975] loss: 0.4659409523010254
[step: 976] loss: 0.45714521408081

[step: 144] loss: 36.8775634765625
[step: 145] loss: 36.82897186279297
[step: 146] loss: 36.78010559082031
[step: 147] loss: 36.73091125488281
[step: 148] loss: 36.68134307861328
[step: 149] loss: 36.63135528564453
[step: 150] loss: 36.58087921142578
[step: 151] loss: 36.52984619140625
[step: 152] loss: 36.47819519042969
[step: 153] loss: 36.42582702636719
[step: 154] loss: 36.372711181640625
[step: 155] loss: 36.31881332397461
[step: 156] loss: 36.26410675048828
[step: 157] loss: 36.208580017089844
[step: 158] loss: 36.15221405029297
[step: 159] loss: 36.095008850097656
[step: 160] loss: 36.03691864013672
[step: 161] loss: 35.97791290283203
[step: 162] loss: 35.917930603027344
[step: 163] loss: 35.85694885253906
[step: 164] loss: 35.79490661621094
[step: 165] loss: 35.73175048828125
[step: 166] loss: 35.667449951171875
[step: 167] loss: 35.601966857910156
[step: 168] loss: 35.53527069091797
[step: 169] loss: 35.467342376708984
[step: 170] loss: 35.398155212402344
[step: 171] loss: 35.

[step: 368] loss: 19.397876739501953
[step: 369] loss: 20.04914093017578
[step: 370] loss: 19.741514205932617
[step: 371] loss: 18.994300842285156
[step: 372] loss: 18.27975082397461
[step: 373] loss: 18.695247650146484
[step: 374] loss: 19.116695404052734
[step: 375] loss: 18.35279083251953
[step: 376] loss: 18.167510986328125
[step: 377] loss: 18.611324310302734
[step: 378] loss: 18.246286392211914
[step: 379] loss: 17.937503814697266
[step: 380] loss: 18.196910858154297
[step: 381] loss: 18.058807373046875
[step: 382] loss: 17.771461486816406
[step: 383] loss: 17.875484466552734
[step: 384] loss: 17.868480682373047
[step: 385] loss: 17.63646125793457
[step: 386] loss: 17.60134506225586
[step: 387] loss: 17.664196014404297
[step: 388] loss: 17.53753089904785
[step: 389] loss: 17.388795852661133
[step: 390] loss: 17.41407012939453
[step: 391] loss: 17.416757583618164
[step: 392] loss: 17.273893356323242
[step: 393] loss: 17.17082405090332
[step: 394] loss: 17.1684627532959
[step: 395]

[step: 602] loss: 8.222126960754395
[step: 603] loss: 8.171756744384766
[step: 604] loss: 8.125272750854492
[step: 605] loss: 8.092018127441406
[step: 606] loss: 8.040943145751953
[step: 607] loss: 7.994179725646973
[step: 608] loss: 7.960443496704102
[step: 609] loss: 7.915337562561035
[step: 610] loss: 7.864945888519287
[step: 611] loss: 7.827633380889893
[step: 612] loss: 7.789102554321289
[step: 613] loss: 7.741637229919434
[step: 614] loss: 7.697465896606445
[step: 615] loss: 7.660174369812012
[step: 616] loss: 7.6189165115356445
[step: 617] loss: 7.573670387268066
[step: 618] loss: 7.531538963317871
[step: 619] loss: 7.493630409240723
[step: 620] loss: 7.453948020935059
[step: 621] loss: 7.410836219787598
[step: 622] loss: 7.3679914474487305
[step: 623] loss: 7.328219413757324
[step: 624] loss: 7.289896011352539
[step: 625] loss: 7.250720977783203
[step: 626] loss: 7.210028648376465
[step: 627] loss: 7.169411659240723
[step: 628] loss: 7.130028247833252
[step: 629] loss: 7.092294

[step: 831] loss: 3.2859177589416504
[step: 832] loss: 3.2847561836242676
[step: 833] loss: 3.2874016761779785
[step: 834] loss: 3.287259340286255
[step: 835] loss: 3.2908732891082764
[step: 836] loss: 3.292680025100708
[step: 837] loss: 3.3131113052368164
[step: 838] loss: 3.3322219848632812
[step: 839] loss: 3.3987631797790527
[step: 840] loss: 3.441761016845703
[step: 841] loss: 3.5883026123046875
[step: 842] loss: 3.6212830543518066
[step: 843] loss: 3.81996750831604
[step: 844] loss: 3.713852882385254
[step: 845] loss: 3.742431402206421
[step: 846] loss: 3.472991943359375
[step: 847] loss: 3.292815685272217
[step: 848] loss: 3.1338913440704346
[step: 849] loss: 3.083726406097412
[step: 850] loss: 3.1270699501037598
[step: 851] loss: 3.21067476272583
[step: 852] loss: 3.3202035427093506
[step: 853] loss: 3.319389581680298
[step: 854] loss: 3.3194758892059326
[step: 855] loss: 3.1967804431915283
[step: 856] loss: 3.1026341915130615
[step: 857] loss: 3.0219459533691406
[step: 858] lo

In [ ]:
Bayseian(txs, forecastDay, unit)